In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
import polars as pl
import warnings
import os

warnings.filterwarnings('ignore', category=FutureWarning)
sns.set(style="whitegrid")
%matplotlib inline

## Data Preview

In [2]:
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
data_dictionary = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/data_dictionary.csv')

In [3]:
print(f"Train shape: {train.shape}")
train.head()

Train shape: (3960, 82)


,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
print(f"Test shape: {test.shape}")
test.head()

Test shape: (20, 59)


,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,32.6909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,3.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,27.0552,NaN,NaN,Fall,2.340,Fall,46.0,64.0,Summer,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,NaN,NaN,NaN,Summer,2.170,Fall,38.0,54.0,Summer,2.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,45.9966,NaN,NaN,Winter,2.451,Summer,31.0,45.0,Winter,0.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,Summer,1.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Age: `Basic_Demos-Age`

Sex: `Basic_Demos-Sex`

Internet Usage: `PreInt_EduHx-computerinternet_hoursday`

CGAS score: `CGAS-CGAS_Score`    (đánh giá chức năng tổng quát của trẻ em, điểm càng thấp càng có vấn đề => mình nghĩ đây là một feature quan trọng)

Physical: `Physical-BMI`, `Physical-Height`, `Physical-Weight`, `Physical-Waist_Circumference`, `Physical-Diastolic_BP`, `Physical-HeartRate`, `Physical-Systolic_BP` (các thang đo sức khỏe vật lý) 

FGC: `Fitness_Endurance-Max_Stage`, `Fitness_Endurance-Time_Mins`, `Fitness_Endurance-Time_Sec`, `FGC-FGC_CU`, `FGC-FGC_CU_Zone`, `FGC-FGC_GSND`,
       `FGC-FGC_GSND_Zone`, `FGC-FGC_GSD`, `FGC-FGC_GSD_Zone`, `FGC-FGC_PU`,
       `FGC-FGC_PU_Zone`, `FGC-FGC_SRL`, `FGC-FGC_SRL_Zone`, `FGC-FGC_SRR`,
       `FGC-FGC_SRR_Zone`, `FGC-FGC_TL`, `FGC-FGC_TL_Zone`
       (các thang đo đánh giá tim mạch, sức mạnh cơ bắp, linh hoạt qua các bài kiểm tra sức bền thể chất)


BIA:  'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
       'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
       'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
       'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
       'BIA-BIA_TBW' (theo dõi các yếu tố thành phần cơ thể như nước, mỡ, cơ bắp)

SDS: đánh giá rối loạn giấc ngủ ở trẻ

PAQ: đánh giá mức độ tham gia các hoạt động thể chất trong 10 ngày gần nhất

PCIAT: thang đo hành vi và đặc tính khi sử dụng Internet quá mức (gồm 20 câu hỏi)Age: `Basic_Demos-Age`

Sex: `Basic_Demos-Sex`

Internet Usage: `PreInt_EduHx-computerinternet_hoursday`

CGAS score: `CGAS-CGAS_Score`    (đánh giá chức năng tổng quát của trẻ em, điểm càng thấp càng có vấn đề => mình nghĩ đây là một feature quan trọng)

Physical: `Physical-BMI`, `Physical-Height`, `Physical-Weight`, `Physical-Waist_Circumference`, `Physical-Diastolic_BP`, `Physical-HeartRate`, `Physical-Systolic_BP` (các thang đo sức khỏe vật lý) 

FGC: `Fitness_Endurance-Max_Stage`, `Fitness_Endurance-Time_Mins`, `Fitness_Endurance-Time_Sec`, `FGC-FGC_CU`, `FGC-FGC_CU_Zone`, `FGC-FGC_GSND`,
       `FGC-FGC_GSND_Zone`, `FGC-FGC_GSD`, `FGC-FGC_GSD_Zone`, `FGC-FGC_PU`,
       `FGC-FGC_PU_Zone`, `FGC-FGC_SRL`, `FGC-FGC_SRL_Zone`, `FGC-FGC_SRR`,
       `FGC-FGC_SRR_Zone`, `FGC-FGC_TL`, `FGC-FGC_TL_Zone`
       (các thang đo đánh giá tim mạch, sức mạnh cơ bắp, linh hoạt qua các bài kiểm tra sức bền thể chất)


BIA:  'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
       'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
       'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
       'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
       'BIA-BIA_TBW' (theo dõi các yếu tố thành phần cơ thể như nước, mỡ, cơ bắp)

SDS: đánh giá rối loạn giấc ngủ ở trẻ

PAQ: đánh giá mức độ tham gia các hoạt động thể chất trong 10 ngày gần nhất

PCIAT: thang đo hành vi và đặc tính khi sử dụng Internet quá mức (gồm 20 câu hỏi)

---
## Baseline

In [5]:
# Đầu tiên loại bỏ các row với `sii` mang giá trị NaN 
print(f"Before drop: #rows = {train.shape[0]}")
train = train.dropna(subset=['sii'])
print(f"After drop: #rows = {train.shape[0]}")

Before drop: #rows = 3960
After drop: #rows = 2736


In [6]:
target = train['sii']
train = train.drop(columns=['sii'])

Chuyển giá trị của các cột có dữ liệu ở dạng `categorical` -> `numerical`

In [7]:
train_cat_cols = train.select_dtypes(exclude='number').columns

train[train_cat_cols]


,id,Basic_Demos-Enroll_Season,CGAS-Season,Physical-Season,Fitness_Endurance-Season,FGC-Season,BIA-Season,PAQ_A-Season,PAQ_C-Season,PCIAT-Season,SDS-Season,PreInt_EduHx-Season
0,00008ff9,Fall,Winter,Fall,NaN,Fall,Fall,NaN,NaN,Fall,NaN,Fall
1,000fd460,Summer,NaN,Fall,NaN,Fall,Winter,NaN,Fall,Fall,Fall,Summer
2,00105258,Summer,Fall,Fall,Fall,Fall,NaN,NaN,Summer,Fall,Fall,Summer
3,00115b9f,Winter,Fall,Summer,Summer,Summer,Summer,NaN,Winter,Summer,Summer,Winter
5,001f3379,Spring,Winter,Summer,NaN,Summer,Summer,NaN,Spring,Summer,Summer,Spring
...,...,...,...,...,...,...,...,...,...,...,...,...
3953,ff6c2bb8,Fall,NaN,Fall,NaN,Fall,Fall,NaN,Fall,Fall,Fall,Fall
3954,ff759544,Summer,NaN,Summer,NaN,Summer,Fall,NaN,NaN,Summer,Summer,Summer
3955,ff8a2de4,Fall,Spring,Fall,NaN,Fall,Fall,NaN,Winter,Winter,Winter,Fall
3957,ffcd4dbd,Fall,Spring,Winter,NaN,Winter,Winter,NaN,Winter,Winter,Winter,Fall


In [8]:
for col in train_cat_cols: 
    train[col].replace({'Spring':1,'Summer':2,'Fall':3,'Winter':4},inplace=True)
train

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday
0,00008ff9,3,5,0,4.0,51.0,3.0,16.877316,46.0,50.8,...,4.0,4.0,2.0,4.0,55.0,NaN,NaN,NaN,3.0,3.0
1,000fd460,2,9,0,NaN,NaN,3.0,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,0.0,3.0,46.0,64.0,2.0,0.0
2,00105258,2,10,1,3.0,71.0,3.0,16.648696,56.5,75.6,...,2.0,2.0,1.0,1.0,28.0,3.0,38.0,54.0,2.0,2.0
3,00115b9f,4,9,0,3.0,71.0,2.0,18.292347,56.0,81.6,...,4.0,3.0,4.0,1.0,44.0,2.0,31.0,45.0,4.0,0.0
5,001f3379,1,13,1,4.0,50.0,2.0,22.279952,59.5,112.2,...,3.0,1.0,2.0,1.0,34.0,2.0,40.0,56.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3953,ff6c2bb8,3,8,0,NaN,NaN,3.0,17.139810,52.5,67.2,...,0.0,2.0,2.0,1.0,22.0,3.0,41.0,58.0,3.0,2.0
3954,ff759544,2,7,1,NaN,NaN,2.0,13.927006,48.5,46.6,...,3.0,3.0,3.0,0.0,33.0,2.0,48.0,67.0,2.0,0.0
3955,ff8a2de4,3,13,0,1.0,60.0,3.0,16.362460,59.5,82.4,...,2.0,1.0,1.0,0.0,32.0,4.0,35.0,50.0,3.0,1.0
3957,ffcd4dbd,3,11,0,1.0,68.0,4.0,21.441500,60.0,109.8,...,1.0,1.0,0.0,1.0,31.0,4.0,56.0,77.0,3.0,0.0


In [9]:
test_cat_cols = test.select_dtypes(exclude='number').columns

for col in test_cat_cols: 
    test[col].replace({'Spring':1,'Summer':2,'Fall':3,'Winter':4}, inplace=True)

In [10]:
test

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday
0,00008ff9,3,5,0,4.0,51.0,3.0,16.877316,46.00,50.8,...,32.6909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0
1,000fd460,2,9,0,NaN,NaN,3.0,14.035590,48.00,46.0,...,27.0552,NaN,NaN,3.0,2.340,3.0,46.0,64.0,2.0,0.0
2,00105258,2,10,1,3.0,71.0,3.0,16.648696,56.50,75.6,...,NaN,NaN,NaN,2.0,2.170,3.0,38.0,54.0,2.0,2.0
3,00115b9f,4,9,0,3.0,71.0,2.0,18.292347,56.00,81.6,...,45.9966,NaN,NaN,4.0,2.451,2.0,31.0,45.0,4.0,0.0
4,0016bb22,1,18,1,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,2.0,1.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,001f3379,1,13,1,4.0,50.0,2.0,22.279952,59.50,112.2,...,63.1265,NaN,NaN,1.0,4.110,2.0,40.0,56.0,1.0,0.0
6,0038ba98,3,10,0,NaN,NaN,3.0,19.660760,55.00,84.6,...,47.2211,NaN,NaN,4.0,3.670,4.0,27.0,40.0,3.0,3.0
7,0068a485,3,10,1,NaN,NaN,3.0,16.861286,59.25,84.2,...,50.4767,NaN,NaN,3.0,1.270,NaN,NaN,NaN,3.0,2.0
8,0069fbed,2,15,0,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0
9,0083e397,2,19,1,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Remove columns not in test set

In [11]:
train_only_cols = list(set(train.columns) - set(test.columns))
train_only_cols

['PCIAT-PCIAT_14',
 'PCIAT-PCIAT_20',
 'PCIAT-PCIAT_19',
 'PCIAT-PCIAT_17',
 'PCIAT-PCIAT_12',
 'PCIAT-PCIAT_13',
 'PCIAT-PCIAT_05',
 'PCIAT-PCIAT_04',
 'PCIAT-PCIAT_10',
 'PCIAT-PCIAT_07',
 'PCIAT-PCIAT_16',
 'PCIAT-PCIAT_08',
 'PCIAT-PCIAT_06',
 'PCIAT-PCIAT_15',
 'PCIAT-PCIAT_18',
 'PCIAT-PCIAT_Total',
 'PCIAT-PCIAT_02',
 'PCIAT-PCIAT_01',
 'PCIAT-PCIAT_11',
 'PCIAT-PCIAT_03',
 'PCIAT-Season',
 'PCIAT-PCIAT_09']

In [12]:
train.drop(columns=train_only_cols, inplace=True) 
train

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday
0,00008ff9,3,5,0,4.0,51.0,3.0,16.877316,46.0,50.8,...,32.6909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0
1,000fd460,2,9,0,NaN,NaN,3.0,14.035590,48.0,46.0,...,27.0552,NaN,NaN,3.0,2.340,3.0,46.0,64.0,2.0,0.0
2,00105258,2,10,1,3.0,71.0,3.0,16.648696,56.5,75.6,...,NaN,NaN,NaN,2.0,2.170,3.0,38.0,54.0,2.0,2.0
3,00115b9f,4,9,0,3.0,71.0,2.0,18.292347,56.0,81.6,...,45.9966,NaN,NaN,4.0,2.451,2.0,31.0,45.0,4.0,0.0
5,001f3379,1,13,1,4.0,50.0,2.0,22.279952,59.5,112.2,...,63.1265,NaN,NaN,1.0,4.110,2.0,40.0,56.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3953,ff6c2bb8,3,8,0,NaN,NaN,3.0,17.139810,52.5,67.2,...,36.7181,NaN,NaN,3.0,3.440,3.0,41.0,58.0,3.0,2.0
3954,ff759544,2,7,1,NaN,NaN,2.0,13.927006,48.5,46.6,...,30.0453,NaN,NaN,NaN,NaN,2.0,48.0,67.0,2.0,0.0
3955,ff8a2de4,3,13,0,1.0,60.0,3.0,16.362460,59.5,82.4,...,52.8320,NaN,NaN,4.0,3.260,4.0,35.0,50.0,3.0,1.0
3957,ffcd4dbd,3,11,0,1.0,68.0,4.0,21.441500,60.0,109.8,...,54.4630,NaN,NaN,4.0,2.729,4.0,56.0,77.0,3.0,0.0


In [13]:
train.drop(columns=['id'],inplace=True)

test_ids = test['id']
test.drop(columns=['id'],inplace=True)

In [14]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(iterations=100, verbose=0)
model.fit(train, target)

pred = model.predict(test)


In [15]:
sub = pd.DataFrame(test_ids)
sub['sii'] = pred.astype(int)
sub

,id,sii
0,00008ff9,2
1,000fd460,0
2,00105258,0
3,00115b9f,1
4,0016bb22,0
5,001f3379,1
6,0038ba98,0
7,0068a485,0
8,0069fbed,0
9,0083e397,0


In [16]:
sub.to_csv('submission.csv', index=False)